In [1]:
import os
import pandas as pd
import re
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from pathlib import Path

# Step 1: Define element name mapping to ensure column name consistency
ELEMENT_MAPPING = {
    "N": "Nitrogen", "Nitrogen": "Nitrogen",
    "P": "Phosphorus", "Phosphorous": "Phosphorus",
    "K": "Potassium", "Ca": "Calcium",
    "Mg": "Magnesium", "S": "Sulfur",
    "Mn": "Manganese", "B": "Boron",
    "Zn": "Zinc", "Fe": "Iron",
    "Cu": "Copper", "Al": "Aluminum",
    "Aluminium": "Aluminum", "Na": "Sodium",
    "Cl": "Chloride"
}


def standardize_columns(df):
    """
    Standardizes column names by removing special characters,
    trimming whitespace, and mapping chemical element names to a consistent format.
    """
    new_columns = []
    
    for col in df.columns:
        clean_col = col.strip()  # Remove leading and trailing spaces
        clean_col = re.sub(r"[()%\-_/]", "", clean_col)  # Remove special characters
        
        # Standardize sample ID column name
        if clean_col.lower() in ["sample.id", "sampleid"]:
            std_col = "Sample_ID"
        # Retain spectral column names (e.g., 500nm, 600nm)
        elif re.match(r"^\d+(\.\d+)?nm$", clean_col.lower()):
            std_col = clean_col
        else:
            std_col = ELEMENT_MAPPING.get(clean_col, clean_col)  # Map element names to a standard format
                
        new_columns.append(std_col)

    df.columns = new_columns
    return df


def remove_outliers_iqr(df, columns):
    """
    Remove outliers using the IQR method and replace them with the median.
    """
    outlier_count = 0  # Record total number of outliers
    modified_rows = set()  # Track rows that have at least one replaced value
    df = df.copy()
    
    for col in columns:
        Q1 = df[col].quantile(0.25)  # First quartile (25th percentile)
        Q3 = df[col].quantile(0.75)  # Third quartile (75th percentile)
        IQR = Q3 - Q1  # Interquartile range (IQR)
        lower_bound = Q1 - 1.5 * IQR  # Lower bound
        upper_bound = Q3 + 1.5 * IQR  # Upper bound
        
        # Identify outliers
        outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
        outlier_count += outliers.sum()
        modified_rows.update(df.index[outliers])
        
        # Replace outliers with the median of the column
        median_value = df[col].median()
        df.loc[outliers, col] = median_value
    
    return df, outlier_count, len(modified_rows)


# Step 2: Set file paths for input and output data
DATA_ROOT = Path("data/DSMP_Prob_Dalhousie_2025 - data/original_data")
IMPUTED_DATA_ROOT = "data/DSMP_Prob_Dalhousie_2025 - data/imputed_data"

# Ensure output directories exist
os.makedirs(f"{IMPUTED_DATA_ROOT}/DRIED", exist_ok=True)
os.makedirs(f"{IMPUTED_DATA_ROOT}/FRESH", exist_ok=True)

# Step 3: Define elements with low missing rates (<10%) to be imputed using the median
LOW_MISSING_ELEMENTS = ["Aluminum", "Boron", "Nitrogen"]

# Step 4: Process raw data files
for sample_type in ["DRIED", "FRESH"]:
    sample_path = DATA_ROOT / sample_type
    
    for file in sample_path.glob("*.csv"):
        df = pd.read_csv(file)  # Load CSV file
        df = standardize_columns(df)  # Standardize column names

        # Step 5: Remove invalid rows
        df.dropna(how="all", subset=df.columns.difference(["Sample_ID"]), inplace=True)

        # Step 6: Identify numeric columns for processing
        numeric_cols = df.select_dtypes(include=["number"]).columns

        # Step 7: Fill missing values for low missing-rate elements using the median
        for element in LOW_MISSING_ELEMENTS:
            if element in df.columns:
                df[element] = df[element].fillna(df[element].median())

        # Step 8: Standardize numerical data before applying KNN imputation
        scaler = StandardScaler()
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

        # Step 9: Apply KNN imputation to fill missing values in high-missing-rate columns
        knn_imputer = KNNImputer(n_neighbors=5)
        df[numeric_cols] = knn_imputer.fit_transform(df[numeric_cols])

        # Step 10: Reverse standardization to restore original scale
        df[numeric_cols] = scaler.inverse_transform(df[numeric_cols])

        # Step 11: Apply IQR-based outlier removal
        df, total_outliers, total_modified_rows = remove_outliers_iqr(df, numeric_cols)
        print(f"Total outliers replaced: {total_outliers}")
        print(f"Total rows with replaced values: {total_modified_rows}")

        # Extract season number from filename for better categorization
        match = re.search(r"season[-_]?(\d+)", file.stem, re.IGNORECASE)
        season_num = int(match.group(1)) if match else "unknown"

        # Define the output file path
        save_path = f"{IMPUTED_DATA_ROOT}/{sample_type}/{sample_type}_season{season_num}_imputed.csv"
        
        # Save the processed data to a new CSV file
        df.to_csv(save_path, index=False)
        print(f"Processed file saved: {save_path}")

print("All missing values have been successfully imputed, outliers handled, and data saved in the 'imputed_data' directory.")


Total outliers replaced: 10374
Total rows with replaced values: 16
Processed file saved: data/DSMP_Prob_Dalhousie_2025 - data/imputed_data/DRIED/DRIED_season1_imputed.csv
Total outliers replaced: 9732
Total rows with replaced values: 54
Processed file saved: data/DSMP_Prob_Dalhousie_2025 - data/imputed_data/DRIED/DRIED_season2_imputed.csv
Total outliers replaced: 16679
Total rows with replaced values: 53
Processed file saved: data/DSMP_Prob_Dalhousie_2025 - data/imputed_data/DRIED/DRIED_season3_imputed.csv
Total outliers replaced: 69084
Total rows with replaced values: 56
Processed file saved: data/DSMP_Prob_Dalhousie_2025 - data/imputed_data/DRIED/DRIED_season4_imputed.csv
Total outliers replaced: 5006
Total rows with replaced values: 13
Processed file saved: data/DSMP_Prob_Dalhousie_2025 - data/imputed_data/FRESH/FRESH_season1_imputed.csv
Total outliers replaced: 6270
Total rows with replaced values: 60
Processed file saved: data/DSMP_Prob_Dalhousie_2025 - data/imputed_data/FRESH/FRE